# CONVERTENDO AS CURVAS (CANAIS) DOS ARQUIVOS DLIS EM DATAFRAMES

### IMPORTANDO AS BIBLIOTECAS NECESSÁRIAS

In [ ]:
# import glob
from dlisio import dlis
import pandas as pd
import re
from collections import Iterable
import numpy as np

In [2]:
pd.set_option('display.max_rows', 7000)

### MAIS A SOBRE DLISIO: BIBLIOTECA PARA LEITURA E ANÁLISE DE DADOS DOS ARQUIVOS .DLIS:

https://dlisio.readthedocs.io/en/latest/

### FUNÇÕES UTILIZADAS

#### FUNÇÃO PARA REMOVER VALORES REPETIDOS DE UMA LISTA

In [2]:
def remove_repetidos(lista):
    l = []
    for i in lista:
        if i not in l:
            l.append(i)
    l.sort()
    return l

#### FUNÇÃO PARA DESENCADEAR LISTAS

In [3]:
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item

#### FUNÇÃO QUE CRIA UM DATAFRAME CONTENDO PARÂMETROS DOS DLIS

In [4]:
def summary_dataframe(object, **kwargs):
    # Create an empty dataframe
    df = pd.DataFrame()
    
    # Iterate over each of the keyword arguments
    for i, (key, value) in enumerate(kwargs.items()):
        list_of_values = []
        
        # Iterate over each parameter and get the relevant key
        for item in object:
            # Account for any missing values.
            try:
                x = getattr(item, key)
                list_of_values.append(x)
            except:
                list_of_values.append('')
                continue
        
        # Add a new column to our data frame
        df[value]=list_of_values
    
    # Sort the dataframe by column 1 and return it
    return df.sort_values(df.columns[0])

#### FUNÇÃO PARA FILTRAR A PALAVRA ESCOLHIDA (PARÂMETRO FÍSICO) E RETORNAR UMA LISTA COM O NOME DOS CANAIS DO RESPECTIVO PARÂMETRO

In [6]:
def filtro(dataframe, nome_canal):
    df_novo = dataframe[dataframe['Long Name'].str.contains(nome_canal)]
    lista = df_novo['Name'].to_list()
    return lista

### VARIÁVEIS ENCONTRADAS NO NOTEBOOK

**dicio_dlis:** dicionário que contém todos os arquivos dlis da pasta selecionada

**lista_logicos1:** lista encadeada de todos os lógicos dos arquivos dlis

**lista_logicos2:** lista de todos os lógicos dos arquivos dlis

**lista_frames:** lista de todos os frames dos dlis

**lista_frames2:** lista sem nomes repetidos de todos os frames dos dlis

**lista_sumario:** lista contendo os dataframes com os parâmetros dos dlis

**channels:** dataframe com os parâmetros dos dlis

**propriedades:** lista onde o usuário coloca as propriedades físicas das curvas do filtro

**lista_alvo:** lista com o nome dos canais filtrados

**profundidade:** lista onde o usuário coloca uma string do Long Name da profundidade

**lista_profundidade:** lista com o nome dos canais de profundidade filtrados a partir da string anterior

**lista_curvas:** lista de dataframes, onde cada dataframe é uma curva (canal)

**lista_curvas2:** lista com os mesmos dataframes, mas sem os valores constantes

**curvas_alvo:** dataframe contendo todas as curvas com a profundidade indexada

**curvas_sem_indice:** dataframe contendo todas as curvas com a profundidade como coluna 


### LENDO OS ARQUIVOS DLIS

In [7]:
#colocar o nome da pasta, em formato de string, onde os arquivos dlis se encontram
path = 'dlis'

In [8]:
lista_dlis = []
lista_read = []

#importando todos os arquivos dlis da pasta '
for v, path_dlis in enumerate(glob.glob(f'{path}**/*dlis')):
    lista_dlis.append(f'dlis_{v}')
    lista_read.append(path_dlis)

#dicionário com todos os arquivos dlis que se encontram na pasta
dicio_dlis = dict(zip(lista_dlis, lista_read))

In [9]:
dicio_dlis

{'dlis_0': 'dlis\\1APS_0029__AP_1APS_0029__AP.dlis'}

In [10]:
#lendo os arquivos dlis com a biblioteca dlisio
lista_logicos1 = []
for i, v in dicio_dlis.items():
    f, *tail = dlis.load(dicio_dlis[i])
    tail.append(f)
    #colocando todos os arquivos lógicos na lista: lista_logicos1 
    lista_logicos1.append(tail)

In [11]:
lista_logicos1

[[LogicalFile(1APS 0029  APBRA1APS 0029  AP2),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP3),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP4),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP5),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP6),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP7),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP8),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP9),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP10),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP11),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP12),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP13),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP14),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP15),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP16),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP17),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP18),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP19),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP20),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP21),
  LogicalFile(1APS 0029  APBRA1APS 0029  AP22),


In [12]:
#utilizando a função para desencadear a lista
lista_logicos2 = list(flatten(lista_logicos1))

### EXIBINDO O CABEÇALHO DOS 4 PRIMEIROS ARQUIVOS LÓGICOS DA LISTA_LÓGICOS2

In [54]:
origin, *origin_tail = lista_logicos2[0].origins
#ver de qual dlis são esses logicos

In [55]:
origin.describe()

------
Origin
------
name   : DEFINING-ORIGIN
origin : 0
copy   : 0

Logical file ID          : 1APS 0029  APBRA1APS 0029  AP2
File set name and number : SPEP-AP-01C1 / 37990
File number and type     : 169414237 / RECALL-DATA

Field                   : LIS_2000/11/11:17:53:
Well (id/name)          : 1APS 0029  AP / 1APS 0029  AP
Produced by (code/name) : 150 /
Produced for            : PETROBRAS
Created                 : 2019-11-11 17:04:41

Created by              : PETROBANK, (version: 4)
Other programs/services : XFER_DLWRIT 1.00


In [56]:
origin2, *origin_tail2 = lista_logicos2[1].origins
origin2.describe()

------
Origin
------
name   : DEFINING-ORIGIN
origin : 0
copy   : 0

Logical file ID          : 1APS 0029  APBRA1APS 0029  AP3
File set name and number : SPEP-AP-01C1 / 37990
File number and type     : 169414237 / RECALL-DATA

Field                   : LIS_2000/11/11:17:53:
Well (id/name)          : 1APS 0029  AP / 1APS 0029  AP
Produced by (code/name) : 150 /
Produced for            : PETROBRAS
Created                 : 2019-11-11 17:04:41

Created by              : PETROBANK, (version: 4)
Other programs/services : XFER_DLWRIT 1.00


In [57]:
origin3, *origin_tail3 = lista_logicos2[1].origins
origin3.describe()

------
Origin
------
name   : DEFINING-ORIGIN
origin : 0
copy   : 0

Logical file ID          : 1APS 0029  APBRA1APS 0029  AP3
File set name and number : SPEP-AP-01C1 / 37990
File number and type     : 169414237 / RECALL-DATA

Field                   : LIS_2000/11/11:17:53:
Well (id/name)          : 1APS 0029  AP / 1APS 0029  AP
Produced by (code/name) : 150 /
Produced for            : PETROBRAS
Created                 : 2019-11-11 17:04:41

Created by              : PETROBANK, (version: 4)
Other programs/services : XFER_DLWRIT 1.00


In [58]:
#A partir dessa quarto lógico, o cabeçalho é o mesmo.
origin4, *origin_tail4 = lista_logicos2[2].origins
origin4.describe()

------
Origin
------
name   : DEFINING-ORIGIN
origin : 0
copy   : 0

Logical file ID          : 1APS 0029  APBRA1APS 0029  AP4
File set name and number : SPEP-AP-01C1 / 37990
File number and type     : 169414237 / RECALL-DATA

Field                   : LIS_2000/11/11:17:53:
Well (id/name)          : 1APS 0029  AP / 1APS 0029  AP
Produced by (code/name) : 150 /
Produced for            : PETROBRAS
Created                 : 2019-11-11 17:04:41

Created by              : PETROBANK, (version: 4)
Other programs/services : XFER_DLWRIT 1.00


### EXTRAINDO OS FRAMES DOS ARQUIVOS LÓGICOS

In [13]:
lista_frames = []
for i in lista_logicos2:
    for fr in i.frames:
        lista_frames.append(fr)

In [14]:
#manipulação para remover os frames duplicados sem alterar o type
df_frames = pd.DataFrame(lista_frames, index = lista_frames)
df_frames.index = df_frames.index.astype('string')
df_frames = df_frames[~df_frames.index.duplicated(keep = 'first')]

In [15]:
lista_frames2 = df_frames[0].to_list()

In [16]:
lista_frames2

[Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,CAL/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]),
 Frame([0,1,HDT/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,FDC/CNL/CNL/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,GR/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,HDT/HDT/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]),
 Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]),
 Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]),
 Frame([0,1,INTEGRACAO_DO_SONICO/BOREHOLE-DEPTH/1/.20/M]),
 Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M])]

### FILTRANDO OS CANAIS NECESSÁRIOS 

Como a quantidade de curvas dentro dos arquivos dlis é muito grande, para diminuir a demanda computacional, é necessário filtrar as curvas que realmente serão utilizadas no projeto.

#### APLICANDO A FUNÇÃO SUMMARY_DATAFRAME E CRIANDO UM DATAFRAME COM OS PARÂMETROS DOS CANAIS: LONG NAME, DIMENSION, UNITS, FRAME.

A partir do parâmetro "Long Name", poderemos filtrar os nomes dos canais pelo parâmetro físico que ele representa.

In [17]:
#aplicando a função summary dataframe
lista_sumario = []
for i in lista_logicos2:
    lista_sumario.append(summary_dataframe(i.channels, name='Name', long_name='Long Name', dimension='Dimension', units='Units', frame='Frame'))
    

In [18]:
#concatenando todos os dataframes em somente um
channels = pd.concat(lista_sumario)

In [19]:
channels

,Name,Long Name,Dimension,Units,Frame
0,DUMM,DUMM,[1],None,"Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M])"
1,INDEX441,INDEX441,[1],m,"Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M])"
0,DUMM,DUMM,[1],None,"Frame([0,1,CAL/BOREHOLE-DEPTH/1/1.0/M])"
1,INDEX441,INDEX441,[1],m,"Frame([0,1,CAL/BOREHOLE-DEPTH/1/1.0/M])"
2,CIL,Unrecognised,[1],None,"Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M])"
...,...,...,...,...,...
2,TOT_3,Sonic Transit Time (Slowness),[1],None,"Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M])"
2,CIL,Unrecognised,[1],None,"Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M])"
3,INDEX440,INDEX440,[1],m,"Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M])"
0,RILD,Deep Resistivity,[1],None,"Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M])"


#### FILTRANDO OS CANAIS.

Primeiramente, colocamos o nome da(s) propriedade(s) escolhida(s) na lista abaixo. Esse nome pode ser conferido no dataframe acima (Ctrl+f para verificar ortografia, letras maiúsculas e minúsculas, etc.)

In [20]:
#lista que deve conter os nomes das propriedades físicas que o usuário deseja salvar
propriedades = ['Gamma']

Após isso, utilizamos a função filtra_canais para filtrarmos os nomes dos canais que contém a propriedade escolhida e colocamos os resultados na lista_alvo.

In [21]:
lista_alvo = []
for i in propriedades:
    #filra_canais(nome do dataframe com os parâmetros definida acima, i = cada propriedade da lista "propriedades")
    lista_alvo.append(filtro(channels, i))

In [22]:
#utilizando a função para desencadear a lista
lista_alvo =  list(flatten(lista_alvo))

In [23]:
lista_alvo

['GR', 'GR', 'GR', 'GR']

In [24]:
#utilizando a função para remover os nomes repetidos
lista_alvo = remove_repetidos(sorted(lista_alvo))

In [25]:
lista_alvo

['GR']

#### FILTRANDO A PROFUNDIDADE

A variável profundidade pode vir com diversos nomes, como: TDEP, DEPTH, INDEX444, etc. Por isso, criamos uma lista que contém uma parte do Long Name (do mesmo modo que na filtragem de canais). Somente analisando o dataframe 'channels', conseguimos saber qual é a variável de profundidade.

In [26]:
#no caso da Bacia do Amazonas, a string 'INDEX' aparece nos long names das profundidades.
profundidade = ['INDEX']

In [29]:
lista_profundidade = []
for i in profundidade:
    #filra_canais(nome do dataframe com os parâmetros definida acima, i = cada propriedade da lista "propriedades")
    lista_profundidade.append(filtro(channels, i))

In [30]:
#aplicando função para desencadear a lista
lista_profundidade = list(flatten(lista_profundidade))

In [31]:
lista_profundidade

['INDEX441',
 'INDEX441',
 'INDEX441',
 'INDEX442',
 'INDEX442',
 'INDEX443',
 'INDEX443',
 'INDEX443',
 'INDEX443',
 'INDEX443',
 'INDEX444',
 'INDEX444',
 'INDEX444',
 'INDEX445',
 'INDEX446',
 'INDEX446',
 'INDEX447',
 'INDEX447',
 'INDEX448',
 'INDEX448',
 'INDEX448',
 'INDEX449',
 'INDEX449',
 'INDEX450',
 'INDEX440']

In [32]:
#aplicando a função para remover os repetidos
lista_profundidade= remove_repetidos(sorted(lista_profundidade))

In [34]:
lista_profundidade

['INDEX440',
 'INDEX441',
 'INDEX442',
 'INDEX443',
 'INDEX444',
 'INDEX445',
 'INDEX446',
 'INDEX447',
 'INDEX448',
 'INDEX449',
 'INDEX450']

**OBS1.: CASO O DLIS CONTENHA O CANAL "TDEP", COLOQUE-O DIRETAMENTE NA LISTA_PROFUNDIDADE (NÃO HÁ PALAVRA CHAVE QUE FILTRE O TDEP)**
    
**DESSA FORMA:**

**lista_profundidade = ['TDEP']**

**OBS2.: QUANDO A PROFUNDIDADE FOR 'TDEP', ELA GERALMENTE VEM EM PÉS. A CONVERSÃO DE PÉS PARA METROS É MULTIPLICAR O VALOR POR 0.00254 . ESSA MULTIPLICAÇÃO DEVERÁ SER COLOCADA NO CÓDIGO DE TRANFORMAR CURVAS EM DATAFRAMES ABAIXO (ESTÁ SINALIZADO NA CÉLULA).**

### CÓDIGO QUE TRANSFORMA OS VALORES DOS CANAIS EM DATAFRAME DE ACORDO COM A PROFUNDIDADE.

Para aplicarmos a função, utilizaremos 4 listas: lista_logicos2, lista_frames2, lista_alvo e lista_profundidade.

In [53]:
lista_curvas = []

for i in lista_logicos2:
    
    for fr in lista_frames2:

        try:

            #pegando as curvas de todos os frames
            curves = fr.curves()
            #curves = frame.curves()

        except Exception as err: 

            print(f' *O {i}, {fr} possui o erro: {err}*')
            pass

        for v in lista_alvo:

            curve_name = str(v) + '_' + str(i) + '_' + str(fr) 

            for profundidade in lista_profundidade:    

                try:

                    curvas = (pd.DataFrame(curves[v], columns = [curve_name], index = curves[profundidade]))
                    
                    #CASO A PROFUNDIDADE ESTEJA EM PÉS, UTILIZE:
                    #curvas = (pd.DataFrame(curves[v], columns = [curva_name], index = curves[profundidade]*0.00254))
                    #comente o curvas acima
                    
                    #transformando o valor -999.25 em nulo
                    curvas[curve_name][curvas[curve_name] == -999.250000] = np.nan
                    
                    lista_curvas.append(curvas)
                    
                    #para alertar quais lógicos possuem as curvas
                    print(f'** O {i}, {fr} possui a curva {v}')


                except Exception as e: 

                    print(f' O {i}, {fr}, possui o erro: {e}')
                    pass

 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no f

** O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]) possui a curva GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX446
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX447
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX448
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX449
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX450
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX440
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP2), Frame([0,1,BSL/GR/B

 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O Logic

 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP4), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O Logic

 O LogicalFile(1APS 0029  APBRA1APS 0029  AP5), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP5), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP5), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP5), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP5), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP5), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP5), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP5), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O Logic

...

 O LogicalFile(1APS 0029  APBRA1APS 0029  AP22), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP22), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP22), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP22), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP22), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP22), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP22), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP22), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR


 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR


 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP23), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR


 O LogicalFile(1APS 0029  APBRA1APS 0029  AP24), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP24), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP24), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP24), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP24), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP24), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP24), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP24), Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR


 O LogicalFile(1APS 0029  APBRA1APS 0029  AP1), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP1), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP1), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP1), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP1), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP1), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP1), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O LogicalFile(1APS 0029  APBRA1APS 0029  AP1), Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O Logic

In [54]:
#removendo os valores contantes (erro comum no dado)
lista_curvas2 = []
for i in lista_curvas:
    lista_curvas2.append(i.reset_index().drop_duplicates(subset='index', keep='last').set_index('index').sort_index())

In [55]:
#lista contendo todos os dataframes (cada dataframe é uma curva)
lista_curvas2

[        GR_LogicalFile(1APS 0029  APBRA1APS 0029  AP2)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])
 index                                                                                                
 3656.0                                          75.466888                                            
 3656.2                                          72.092484                                            
 3656.4                                          24.713242                                            
 3656.6                                          22.932465                                            
 3656.8                                          21.986649                                            
 ...                                                   ...                                            
 4382.2                                                NaN                                            
 4382.4                                                NaN               

In [56]:
curvas_alvo = pd.concat(lista_curvas2, axis = 1)

In [57]:
curvas_sem_index = curvas_alvo.reset_index()

In [58]:
curvas_sem_index

,index,"GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP2)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP2)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP3)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP3)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP4)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP4)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP5)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP5)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP6)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])",...,"GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP22)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP22)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP23)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP23)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP24)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP24)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP25)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP25)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP1)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP1)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])"
0,3656.0,75.466888,NaN,75.466888,NaN,75.466888,NaN,75.466888,NaN,75.466888,...,75.466888,NaN,75.466888,NaN,75.466888,NaN,75.466888,NaN,75.466888,NaN
1,3656.2,72.092484,NaN,72.092484,NaN,72.092484,NaN,72.092484,NaN,72.092484,...,72.092484,NaN,72.092484,NaN,72.092484,NaN,72.092484,NaN,72.092484,NaN
2,3656.4,24.713242,NaN,24.713242,NaN,24.713242,NaN,24.713242,NaN,24.713242,...,24.713242,NaN,24.713242,NaN,24.713242,NaN,24.713242,NaN,24.713242,NaN
3,3656.6,22.932465,NaN,22.932465,NaN,22.932465,NaN,22.932465,NaN,22.932465,...,22.932465,NaN,22.932465,NaN,22.932465,NaN,22.932465,NaN,22.932465,NaN
4,3656.8,21.986649,NaN,21.986649,NaN,21.986649,NaN,21.986649,NaN,21.986649,...,21.986649,NaN,21.986649,NaN,21.986649,NaN,21.986649,NaN,21.986649,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12537,1900.2,NaN,72.727661,NaN,72.727661,NaN,72.727661,NaN,72.727661,NaN,...,NaN,72.727661,NaN,72.727661,NaN,72.727661,NaN,72.727661,NaN,72.727661
12538,1900.4,NaN,70.453888,NaN,70.453888,NaN,70.453888,NaN,70.453888,NaN,...,NaN,70.453888,NaN,70.453888,NaN,70.453888,NaN,70.453888,NaN,70.453888
12539,1900.6,NaN,69.688034,NaN,69.688034,NaN,69.688034,NaN,69.688034,NaN,...,NaN,69.688034,NaN,69.688034,NaN,69.688034,NaN,69.688034,NaN,69.688034
12540,1900.8,NaN,70.542679,NaN,70.542679,NaN,70.542679,NaN,70.542679,NaN,...,NaN,70.542679,NaN,70.542679,NaN,70.542679,NaN,70.542679,NaN,70.542679
